In [29]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def load_images(image_dir, num_images=5):
    images = []
    for i, filename in enumerate(os.listdir(image_dir)):
        if i >= num_images:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)
    return images

def segment_image_with_pca(image, n_components=2, k=3):
    if image is None:
        print("Error: Invalid image")
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_values = image_rgb.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    pixel_values_reduced = pca.fit_transform(pixel_values)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(pixel_values_reduced)
    centers = kmeans.cluster_centers_
    
    # Map labels back to the original color space
    centers = pca.inverse_transform(centers)
    centers = np.uint8(centers)
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image_rgb.shape)
    return segmented_image


def calculate_pixel_accuracy(pred, target):
    return np.mean(pred == target)

def compare_with_mask(segmented_image, mask, accuracies):
    ground_truth_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    segmented_image_resized = cv2.resize(segmented_image, (ground_truth_mask.shape[1], ground_truth_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmented_gray = cv2.cvtColor(segmented_image_resized, cv2.COLOR_BGR2GRAY)

    _, ground_truth_binary = cv2.threshold(ground_truth_mask, 127, 1, cv2.THRESH_BINARY)
    _, segmented_binary = cv2.threshold(segmented_gray, 127, 1, cv2.THRESH_BINARY)

    accuracy = calculate_pixel_accuracy(segmented_binary, ground_truth_binary)

    accuracies.append(accuracy)

def plot_results(results):
    # Calculate mean values for each metric
    mean_accuracy = np.mean(results['accuracy'])

    return mean_accuracy


def main():
    k = 3  # Fix k=3 for clustering
    results = {'accuracy': []}

    image_dir = 'Data/Image'
    mask_dir = 'Data/Mask'
    
    images = load_images(image_dir)
    masks = load_images(mask_dir)
    i=0
    for img, mask in zip(images, masks):
        print(f"Image {i+1}")
        segmented_image = segment_image_with_pca(img, k=k)
        
        i+=1
        if segmented_image is not None:
            compare_with_mask(segmented_image, mask, results['accuracy'])
        
    print(f"accuracy = {plot_results(results)}")

if __name__ == "__main__":
    main()


Image 1
Image 2
Image 3
Image 4
accuracy = 0.5792622401793902


In [12]:
import os
import cv2
import numpy as np
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans
import plotly.graph_objects as go

def load_images(image_dir, num_images=5):
    images = []
    for i, filename in enumerate(os.listdir(image_dir)):
        if i >= num_images:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)
    return images

def segment_image_with_ica(image, n_components=2, k=3):
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_values = image_rgb.reshape((-1, 3)).astype(np.float32)

    # Apply ICA for dimensionality reduction
    ica = FastICA(n_components=n_components, random_state=42)
    pixel_values_reduced = ica.fit_transform(pixel_values)

    # K-means clustering on reduced dimensions
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(pixel_values_reduced)

    # Map each cluster label to the mean color in the original image for each cluster
    segmented_image = np.zeros_like(pixel_values)
    for i in range(k):
        cluster_mean_color = np.mean(pixel_values[labels == i], axis=0)
        segmented_image[labels == i] = cluster_mean_color

    # Reshape to the original image dimensions
    segmented_image = segmented_image.reshape(image_rgb.shape)
    segmented_image = np.uint8(segmented_image)  # Convert back to uint8
    return segmented_image


def calculate_iou(pred, target):
    intersection = np.logical_and(pred, target)
    union = np.logical_or(pred, target)
    return np.sum(intersection) / np.sum(union)

def calculate_dice(pred, target):
    intersection = np.logical_and(pred, target)
    return 2. * np.sum(intersection) / (np.sum(pred) + np.sum(target))

def calculate_pixel_accuracy(pred, target):
    return np.mean(pred == target)

def compare_with_mask(segmented_image, mask, iou_scores, dice_scores, accuracies):
    ground_truth_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    segmented_image_resized = cv2.resize(segmented_image, (ground_truth_mask.shape[1], ground_truth_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmented_gray = cv2.cvtColor(segmented_image_resized, cv2.COLOR_BGR2GRAY)

    _, ground_truth_binary = cv2.threshold(ground_truth_mask, 127, 1, cv2.THRESH_BINARY)
    _, segmented_binary = cv2.threshold(segmented_gray, 127, 1, cv2.THRESH_BINARY)

    iou = calculate_iou(segmented_binary, ground_truth_binary)
    dice = calculate_dice(segmented_binary, ground_truth_binary)
    accuracy = calculate_pixel_accuracy(segmented_binary, ground_truth_binary)
    
    iou_scores.append(iou)
    dice_scores.append(dice)
    accuracies.append(accuracy)

def plot_results(results):
    # Calculate mean values for each metric
    mean_accuracy = np.mean(results['accuracy'])
    mean_iou = np.mean(results['iou'])
    mean_dice = np.mean(results['dice'])

    # Create histogram
    fig = go.Figure(data=[
        go.Bar(name='Accuracy', x=['Metrics'], y=[mean_accuracy], marker_color='blue'),
        go.Bar(name='IoU Score', x=['Metrics'], y=[mean_iou], marker_color='green'),
        go.Bar(name='Dice Score', x=['Metrics'], y=[mean_dice], marker_color='red')
    ])

    fig.update_layout(
        title="Segmentation Metrics for K=3 with ICA",
        xaxis_title="Metrics",
        yaxis_title="Score",
        barmode='group',
        height=400,
        width=600
    )
    
    # Save the plot
    if not os.path.exists('Image'):
        os.makedirs('Image')
    fig.write_image("Image/metrics_histogram_with_ica_k3.png")
    
    print("Histogram saved as Image/metrics_histogram_with_ica_k3.png")

def main():
    k = 3  # Fix k=3 for clustering
    results = {'accuracy': [], 'iou': [], 'dice': []}

    image_dir = 'Data/Image'
    mask_dir = 'Data/Mask'
    
    images = load_images(image_dir)
    masks = load_images(mask_dir)
    i = 0
    for img, mask in zip(images, masks):
        print(f"Image {i+1}")   
        segmented_image = segment_image_with_ica(img, k=k)
        i+=1
        if segmented_image is not None:
            compare_with_mask(segmented_image, mask, results['iou'], results['dice'], results['accuracy'])
        
    plot_results(results)

if __name__ == "__main__":
    main()


Image 1
Image 2


c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.

c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.



Image 3


c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.



Image 4


c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.



Histogram saved as Image/metrics_histogram_with_ica_k3.png


In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import plotly.graph_objects as go

def load_images(image_dir, num_images=5):
    images = []
    for i, filename in enumerate(os.listdir(image_dir)):
        if i >= num_images:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)
    return images

def segment_image_without_reduction(image, k=3):
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_values = image_rgb.reshape((-1, 3)).astype(np.float32)

    # Perform K-means clustering directly on RGB pixel values
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(pixel_values)
    centers = np.uint8(kmeans.cluster_centers_)  # The RGB values of each cluster center

    # Map each pixel to the color of its respective cluster center
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image_rgb.shape)
    return segmented_image

def calculate_iou(pred, target):
    intersection = np.logical_and(pred, target)
    union = np.logical_or(pred, target)
    return np.sum(intersection) / np.sum(union)

def calculate_dice(pred, target):
    intersection = np.logical_and(pred, target)
    return 2. * np.sum(intersection) / (np.sum(pred) + np.sum(target))

def calculate_pixel_accuracy(pred, target):
    return np.mean(pred == target)

def compare_with_mask(segmented_image, mask, iou_scores, dice_scores, accuracies):
    ground_truth_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    segmented_image_resized = cv2.resize(segmented_image, (ground_truth_mask.shape[1], ground_truth_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmented_gray = cv2.cvtColor(segmented_image_resized, cv2.COLOR_BGR2GRAY)

    _, ground_truth_binary = cv2.threshold(ground_truth_mask, 127, 1, cv2.THRESH_BINARY)
    _, segmented_binary = cv2.threshold(segmented_gray, 127, 1, cv2.THRESH_BINARY)

    iou = calculate_iou(segmented_binary, ground_truth_binary)
    dice = calculate_dice(segmented_binary, ground_truth_binary)
    accuracy = calculate_pixel_accuracy(segmented_binary, ground_truth_binary)
    
    iou_scores.append(iou)
    dice_scores.append(dice)
    accuracies.append(accuracy)

def plot_results(results):
    mean_accuracy = np.mean(results['accuracy'])
    mean_iou = np.mean(results['iou'])
    mean_dice = np.mean(results['dice'])

    fig = go.Figure(data=[
        go.Bar(name='Accuracy', x=['Metrics'], y=[mean_accuracy], marker_color='blue'),
        go.Bar(name='IoU Score', x=['Metrics'], y=[mean_iou], marker_color='green'),
        go.Bar(name='Dice Score', x=['Metrics'], y=[mean_dice], marker_color='red')
    ])

    fig.update_layout(
        title="Segmentation Metrics for K=3 without Dimensionality Reduction",
        xaxis_title="Metrics",
        yaxis_title="Score",
        barmode='group',
        height=400,
        width=600
    )

    if not os.path.exists('Image'):
        os.makedirs('Image')
    fig.write_image("Image/metrics_histogram_without_reduction_k3.png")

    print("Histogram saved as Image/metrics_histogram_without_reduction_k3.png")

def main():
    k = 3
    results = {'accuracy': [], 'iou': [], 'dice': []}

    image_dir = 'Data/Image'
    mask_dir = 'Data/Mask'
    
    images = load_images(image_dir)
    masks = load_images(mask_dir)
    i = 0
    for img, mask in zip(images, masks):
        print(f"Image {i+1}")
        segmented_image = segment_image_without_reduction(img, k=k)
        i+=1
        if segmented_image is not None:
            compare_with_mask(segmented_image, mask, results['iou'], results['dice'], results['accuracy'])
        
    plot_results(results)

if __name__ == "__main__":
    main()


Histogram saved as Image/metrics_histogram_without_reduction_k3.png


In [25]:
import cv2
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
import os
import plotly.graph_objects as go

def load_images(image_dir, num_images=5):
    images = []
    for i, filename in enumerate(os.listdir(image_dir)):
        if i >= num_images:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)
    return images

def segment_image_meanshift(image):
    if image is None:
        return None
    # Convert image to RGB and reshape to a 2D array of pixels
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_values = image_rgb.reshape((-1, 3)).astype(np.float32)

    # Estimate bandwidth and apply Mean Shift clustering
    bandwidth = estimate_bandwidth(pixel_values, quantile=0.2, n_samples=500)
    meanshift = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    labels = meanshift.fit_predict(pixel_values)
    centers = np.uint8(meanshift.cluster_centers_)

    # Map labels to cluster centers
    segmented_image = centers[labels.flatten()].reshape(image_rgb.shape)
    return segmented_image

def calculate_iou(pred, target):
    intersection = np.logical_and(pred, target)
    union = np.logical_or(pred, target)
    return np.sum(intersection) / np.sum(union)

def calculate_dice(pred, target):
    intersection = np.logical_and(pred, target)
    return 2. * np.sum(intersection) / (np.sum(pred) + np.sum(target))

def calculate_pixel_accuracy(pred, target):
    return np.mean(pred == target)

def compare_with_mask(segmented_image, mask, iou_scores, dice_scores, accuracies):
    ground_truth_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    segmented_image_resized = cv2.resize(segmented_image, (ground_truth_mask.shape[1], ground_truth_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmented_gray = cv2.cvtColor(segmented_image_resized, cv2.COLOR_BGR2GRAY)

    _, ground_truth_binary = cv2.threshold(ground_truth_mask, 127, 1, cv2.THRESH_BINARY)
    _, segmented_binary = cv2.threshold(segmented_gray, 127, 1, cv2.THRESH_BINARY)

    iou = calculate_iou(segmented_binary, ground_truth_binary)
    dice = calculate_dice(segmented_binary, ground_truth_binary)
    accuracy = calculate_pixel_accuracy(segmented_binary, ground_truth_binary)
    
    iou_scores.append(iou)
    dice_scores.append(dice)
    accuracies.append(accuracy)

def plot_results(results):
    # Calculate mean values for each metric
    mean_accuracy = np.mean(results['accuracy'])
    mean_iou = np.mean(results['iou'])
    mean_dice = np.mean(results['dice'])

    # Create histogram
    fig = go.Figure(data=[
        go.Bar(name='Accuracy', x=['Metrics'], y=[mean_accuracy], marker_color='blue'),
        go.Bar(name='IoU Score', x=['Metrics'], y=[mean_iou], marker_color='green'),
        go.Bar(name='Dice Score', x=['Metrics'], y=[mean_dice], marker_color='red')
    ])

    fig.update_layout(
        title="Segmentation Metrics with Mean Shift Clustering",
        xaxis_title="Metrics",
        yaxis_title="Score",
        barmode='group',
        height=400,
        width=600
    )
    
    # Save the plot
    if not os.path.exists('Image'):
        os.makedirs('Image')
    fig.write_image("Image/metrics_histogram_meanshift.png")
    
    print("Histogram saved as Image/metrics_histogram_meanshift.png")

def main():
    results = {'accuracy': [], 'iou': [], 'dice': []}

    image_dir = 'Data/Image'
    mask_dir = 'Data/Mask'
    
    images = load_images(image_dir)
    masks = load_images(mask_dir)
    i = 0
    for img, mask in zip(images, masks):
        print(f"Image {i+1}")   
        segmented_image = segment_image_meanshift(img)
        i+=1
        if segmented_image is not None:
            compare_with_mask(segmented_image, mask, results['iou'], results['dice'], results['accuracy'])
        
    plot_results(results)

if __name__ == "__main__":
    main()


Image 1
Image 2
Image 3
Image 4
Histogram saved as Image/metrics_histogram_meanshift.png


In [27]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA, FastICA
from sklearn.cluster import MeanShift, estimate_bandwidth
import plotly.graph_objects as go

def load_images(image_dir, num_images=5):
    images = []
    for i, filename in enumerate(os.listdir(image_dir)):
        if i >= num_images:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)
    return images

def segment_image_with_pca_meanshift(image, n_components=2):
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_values = image_rgb.reshape((-1, 3)).astype(np.float32)

    # Apply PCA
    pca = PCA(n_components=n_components)
    pixel_values_reduced = pca.fit_transform(pixel_values)

    # Mean Shift clustering
    bandwidth = estimate_bandwidth(pixel_values_reduced, quantile=0.2, n_samples=500)
    meanshift = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    labels = meanshift.fit_predict(pixel_values_reduced)
    centers = pca.inverse_transform(meanshift.cluster_centers_)

    segmented_image = np.uint8(centers[labels.flatten()]).reshape(image_rgb.shape)
    return segmented_image

def segment_image_with_ica_meanshift(image, n_components=2):
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_values = image_rgb.reshape((-1, 3)).astype(np.float32)

    # Apply ICA
    ica = FastICA(n_components=n_components, random_state=42)
    pixel_values_reduced = ica.fit_transform(pixel_values)

    # Mean Shift clustering
    bandwidth = estimate_bandwidth(pixel_values_reduced, quantile=0.2, n_samples=500)
    meanshift = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    labels = meanshift.fit_predict(pixel_values_reduced)
    centers = ica.inverse_transform(meanshift.cluster_centers_)

    segmented_image = np.uint8(centers[labels.flatten()]).reshape(image_rgb.shape)
    return segmented_image

def calculate_iou(pred, target):
    intersection = np.logical_and(pred, target)
    union = np.logical_or(pred, target)
    return np.sum(intersection) / np.sum(union)

def calculate_dice(pred, target):
    intersection = np.logical_and(pred, target)
    return 2. * np.sum(intersection) / (np.sum(pred) + np.sum(target))

def calculate_pixel_accuracy(pred, target):
    return np.mean(pred == target)

def compare_with_mask(segmented_image, mask, iou_scores, dice_scores, accuracies):
    ground_truth_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    segmented_image_resized = cv2.resize(segmented_image, (ground_truth_mask.shape[1], ground_truth_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmented_gray = cv2.cvtColor(segmented_image_resized, cv2.COLOR_BGR2GRAY)

    _, ground_truth_binary = cv2.threshold(ground_truth_mask, 127, 1, cv2.THRESH_BINARY)
    _, segmented_binary = cv2.threshold(segmented_gray, 127, 1, cv2.THRESH_BINARY)

    iou = calculate_iou(segmented_binary, ground_truth_binary)
    dice = calculate_dice(segmented_binary, ground_truth_binary)
    accuracy = calculate_pixel_accuracy(segmented_binary, ground_truth_binary)
    
    iou_scores.append(iou)
    dice_scores.append(dice)
    accuracies.append(accuracy)

def plot_results(results, method_name):
    mean_accuracy = np.mean(results['accuracy'])
    mean_iou = np.mean(results['iou'])
    mean_dice = np.mean(results['dice'])

    fig = go.Figure(data=[
        go.Bar(name='Accuracy', x=['Metrics'], y=[mean_accuracy], marker_color='blue'),
        go.Bar(name='IoU Score', x=['Metrics'], y=[mean_iou], marker_color='green'),
        go.Bar(name='Dice Score', x=['Metrics'], y=[mean_dice], marker_color='red')
    ])

    fig.update_layout(
        title=f"Segmentation Metrics with Mean Shift and {method_name}",
        xaxis_title="Metrics",
        yaxis_title="Score",
        barmode='group',
        height=400,
        width=600
    )

    if not os.path.exists('Image'):
        os.makedirs('Image')
    fig.write_image(f"Image/metrics_histogram_meanshift_{method_name.lower()}.png")
    
    print(f"Histogram saved as Image/metrics_histogram_meanshift_{method_name.lower()}.png")

def main():
    image_dir = 'Data/Image'
    mask_dir = 'Data/Mask'
    images = load_images(image_dir)
    masks = load_images(mask_dir)

    # For PCA
    results_pca = {'accuracy': [], 'iou': [], 'dice': []}
    for img, mask in zip(images, masks):
        segmented_image = segment_image_with_pca_meanshift(img)
        if segmented_image is not None:
            compare_with_mask(segmented_image, mask, results_pca['iou'], results_pca['dice'], results_pca['accuracy'])
    plot_results(results_pca, "PCA")

    # For ICA
    results_ica = {'accuracy': [], 'iou': [], 'dice': []}
    for img, mask in zip(images, masks):
        segmented_image = segment_image_with_ica_meanshift(img)
        if segmented_image is not None:
            compare_with_mask(segmented_image, mask, results_ica['iou'], results_ica['dice'], results_ica['accuracy'])
    plot_results(results_ica, "ICA")

if __name__ == "__main__":
    main()


Histogram saved as Image/metrics_histogram_meanshift_pca.png


c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.

c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.

c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.

c:\Users\ASUS\Miniconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning:

Starting in v1.3, whiten='unit-variance' will be used by default.



Histogram saved as Image/metrics_histogram_meanshift_ica.png
